### RECOMMENDERS

Here we provide the following recommenders:
   - Popularity recommender: the best rated recipes.
   - Collaborative filtering recommender: uses item-item similarities based on users in common, using nearest neighbors searching method for training.
   - Content-based recommender: the similarity between the items recommended is determined by the content of those items rather than learned from user interaction data.

In [ ]:
import numpy as np
import pandas as pd
import csv
import graphlab as gl

In [7]:
df = pd.read_csv("ratings.csv")

In [8]:
df.head(10)

,user,recipe,rating
0,user_1,295440,3
1,user_1,259995,3
2,user_1,261065,3
3,user_2,268660,4
4,user_2,131633,3
5,user_2,summer-berry-eton-mess,3
6,user_2,291087,4
7,user_2,204903,3
8,user_3,124373,3
9,user_3,241588,3


In [10]:
sf = gl.SFrame(data=df)

In [18]:
sf.shape

(60067, 3)

### Popularity recommender

In [15]:
popularity_model = gl.popularity_recommender.create(sf, user_id='user', item_id='recipe', target='rating')

Recsys training: model = popularity

Preparing data set.

Data has 60067 observations with 10000 users and 15218 items.

Data prepared in: 0.122124s

60067 observations to process; with 15218 unique items.

In [26]:
# top rated
user_recipe_rating.groupby(by='recipe')['rating'].mean().sort_values(ascending=False).head(10)  #top 10

recipe
zingy-peach-salsa    4.0
222145               4.0
221157               4.0
221270               4.0
221362               4.0
221463               4.0
221630               4.0
221634               4.0
221700               4.0
222128               4.0
Name: rating, dtype: float64

In [37]:
# get the top for a user
popularity_recomm = popularity_model.recommend(users=["user_2"], k=5)
popularity_recomm.print_rows(num_rows=25)

+--------+-------------------------------+-------+------+
|  user  |             recipe            | score | rank |
+--------+-------------------------------+-------+------+
| user_2 | james-martin-smoked-salmon... |  4.0  |  1   |
| user_2 |             225207            |  4.0  |  2   |
| user_2 |    parmesan-crusted-haddock   |  4.0  |  3   |
| user_2 |             229921            |  4.0  |  4   |
| user_2 |        mushroom-burger        |  4.0  |  5   |
+--------+-------------------------------+-------+------+
[5 rows x 4 columns]



In [48]:
popularity_recomm.save("popularity_recommendations.csv")

### Item-Item recommender

Create a recommender that uses item-item similarities based on users in common.

### Using cosine correlation

In [52]:
itemitem_cos = gl.item_similarity_recommender.create(sf, user_id='user', item_id='recipe', target='rating', 
                                                     similarity_type='cosine')
pred_itemitem_cos = itemitem_cos.predict(sf)
recs_itemitem_cos = itemitem_cos.recommend()     # top 10 collaborative filtering recomendations

Recsys training: model = item_similarity

Preparing data set.

Data has 60067 observations with 10000 users and 15218 items.

Data prepared in: 0.142447s

Training model from provided data.

...

recommendations finished on 6000/10000 queries. users per second: 13831.2

recommendations finished on 7000/10000 queries. users per second: 13770.2

recommendations finished on 8000/10000 queries. users per second: 13703

recommendations finished on 9000/10000 queries. users per second: 13700.3

recommendations finished on 10000/10000 queries. users per second: 13212.5

In [54]:
itemitem_cos.recommend(users=["user_1"], k=10)

user,recipe,score,rank
user_1,218374,0.5,1
user_1,217681,0.452548325062,2
user_1,miguel-barclays-pork-chop-in-mustard-and-l ...,0.424264073372,3
user_1,princess-leia-cinnamon-buns ...,0.399999976158,4
user_1,163499,0.388056993484,5
user_1,149773,0.368229866028,6
user_1,217878,0.342997193336,7
user_1,fish-pie,0.316227793694,8
user_1,steak-caramelised-onion-salad ...,0.296499729156,9
user_1,259234,0.27852421999,10


In [49]:
recs_itemitem_cos.save("itemitem_cos_recommendations.csv")

### Using Pearson correlation

In [55]:
itemitem_cos = gl.item_similarity_recommender.create(sf, user_id='user', item_id='recipe', target='rating', 
                                                     similarity_type='cosine')
pred_itemitem_pearson = itemitem_cos.predict(sf)
recs_itemitem_pearson = itemitem_cos.recommend()     # top 10 collaborative filtering recomendations
recs_itemitem_pearson.save("itemitem_pearson_recommendations.csv")

Recsys training: model = item_similarity

Preparing data set.

Data has 60067 observations with 10000 users and 15218 items.

Data prepared in: 0.131563s

Training model from provided data.

...

recommendations finished on 6000/10000 queries. users per second: 13859.5

recommendations finished on 7000/10000 queries. users per second: 13637

recommendations finished on 8000/10000 queries. users per second: 13326.9

recommendations finished on 9000/10000 queries. users per second: 13115.6

recommendations finished on 10000/10000 queries. users per second: 12564.6